In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class VAE(nn.Module):
    def __init__(self, input_dim=46, latent_dim=16):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 24),
            nn.ReLU(),
            nn.Linear(24, latent_dim),
            nn.ReLU(),
        )
        
        self.mu = nn.Linear(16, latent_dim)
        self.log_var = nn.Linear(16, latent_dim)
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 24),
            nn.ReLU(),
            nn.Linear(24, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
            nn.Sigmoid()  # Use Sigmoid if input values are normalized to [0, 1]
        )
        
    def encode(self, x):
        x = self.encoder(x)
        mu = self.mu(x)
        log_var = self.log_var(x)
        return mu, log_var

    # reparameterize trick
    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        reconstructed = self.decode(z)
        return reconstructed, mu, log_var

def loss_function(reconstructed, original, mu, log_var):
    reconstruction_loss = nn.MSELoss()(reconstructed, original)
    # KL Divergence loss
    kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return reconstruction_loss + kl_loss




In [2]:
from tqdm import tqdm
# training loop
def train_vae(model, dataloader, device, epochs=20,  learning_rate=1e-3):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (data_entries_tensors) in tqdm(enumerate(dataloader), desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            reconstructed, mu, log_var = model(data_entries_tensors)
            loss = loss_function(reconstructed, data_entries_tensors, mu, log_var)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset

class CasasVAEDataset(Dataset):
    def __init__(self, parquet_file):
        # Load the DataFrame from the Parquet file
        self.df = pd.read_parquet(parquet_file).astype(float)

    def __len__(self):
        # Return the total number of samples
        return len(self.df)

    def __getitem__(self, idx):
        # Get the row at the specified index
        row = self.df.iloc[idx]
        sequence = torch.tensor(row, dtype=torch.float32)
        return sequence



In [4]:
###### import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split

# Instantiate and train
vae = VAE(input_dim=53, latent_dim=16)

# Data loaders
parquet_file = './data/vae/combined_sampled_df.parquet'
dataset = CasasVAEDataset(parquet_file)
train_ratio = 0.8
eval_ratio = 0.2

# Wrap them in DataLoaders
train_dataset, eval_dataset = random_split(dataset, [0.8, 0.2])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=64, shuffle=False)
train_vae(vae, train_loader, device, epochs=50)

Epoch 1/50: 0it [00:00, ?it/s]/tmp/ipykernel_2087/4155969802.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sequence = torch.tensor(row, dtype=torch.float32)
Epoch 1/50: 21it [00:00, 101.43it/s]

tensor(110.2939, grad_fn=<MseLossBackward0>)
tensor(68.4847, grad_fn=<MulBackward0>)
tensor(104.7356, grad_fn=<MseLossBackward0>)
tensor(42.0248, grad_fn=<MulBackward0>)
tensor(100.1768, grad_fn=<MseLossBackward0>)
tensor(26.7463, grad_fn=<MulBackward0>)
tensor(109.2446, grad_fn=<MseLossBackward0>)
tensor(22.3226, grad_fn=<MulBackward0>)
tensor(103.8855, grad_fn=<MseLossBackward0>)
tensor(16.4739, grad_fn=<MulBackward0>)
tensor(107.6348, grad_fn=<MseLossBackward0>)
tensor(13.0607, grad_fn=<MulBackward0>)
tensor(113.3737, grad_fn=<MseLossBackward0>)
tensor(12.2381, grad_fn=<MulBackward0>)
tensor(158.2360, grad_fn=<MseLossBackward0>)
tensor(30.8448, grad_fn=<MulBackward0>)
tensor(111.6309, grad_fn=<MseLossBackward0>)
tensor(9.7245, grad_fn=<MulBackward0>)
tensor(101.9161, grad_fn=<MseLossBackward0>)
tensor(8.9782, grad_fn=<MulBackward0>)
tensor(102.6994, grad_fn=<MseLossBackward0>)
tensor(8.2571, grad_fn=<MulBackward0>)
tensor(110.4846, grad_fn=<MseLossBackward0>)
tensor(8.0274, grad_fn=

Epoch 1/50: 44it [00:00, 106.91it/s]

tensor(102.5788, grad_fn=<MseLossBackward0>)
tensor(6.1496, grad_fn=<MulBackward0>)
tensor(106.9641, grad_fn=<MseLossBackward0>)
tensor(5.9037, grad_fn=<MulBackward0>)
tensor(108.6403, grad_fn=<MseLossBackward0>)
tensor(5.8231, grad_fn=<MulBackward0>)
tensor(103.7907, grad_fn=<MseLossBackward0>)
tensor(5.6469, grad_fn=<MulBackward0>)
tensor(110.7134, grad_fn=<MseLossBackward0>)
tensor(5.7015, grad_fn=<MulBackward0>)
tensor(101.1598, grad_fn=<MseLossBackward0>)
tensor(5.3418, grad_fn=<MulBackward0>)
tensor(177.4348, grad_fn=<MseLossBackward0>)
tensor(5.7410, grad_fn=<MulBackward0>)
tensor(108.8206, grad_fn=<MseLossBackward0>)
tensor(5.3285, grad_fn=<MulBackward0>)
tensor(123.1406, grad_fn=<MseLossBackward0>)
tensor(5.1852, grad_fn=<MulBackward0>)
tensor(111.4211, grad_fn=<MseLossBackward0>)
tensor(5.0719, grad_fn=<MulBackward0>)
tensor(264.7565, grad_fn=<MseLossBackward0>)
tensor(5.2753, grad_fn=<MulBackward0>)
tensor(105.3523, grad_fn=<MseLossBackward0>)
tensor(4.8129, grad_fn=<MulBack

Epoch 1/50: 60it [00:00, 104.14it/s]


...

Epoch 33/50: 32it [00:00, 99.28it/s]

tensor(153.3581, grad_fn=<MseLossBackward0>)
tensor(0.0522, grad_fn=<MulBackward0>)
tensor(103.0278, grad_fn=<MseLossBackward0>)
tensor(0.0023, grad_fn=<MulBackward0>)
tensor(113.1070, grad_fn=<MseLossBackward0>)
tensor(0.0258, grad_fn=<MulBackward0>)
tensor(105.4720, grad_fn=<MseLossBackward0>)
tensor(0.0022, grad_fn=<MulBackward0>)
tensor(102.2342, grad_fn=<MseLossBackward0>)
tensor(0.0025, grad_fn=<MulBackward0>)
tensor(98.8203, grad_fn=<MseLossBackward0>)
tensor(0.0026, grad_fn=<MulBackward0>)


Epoch 33/50: 37it [00:00, 97.63it/s]

tensor(313.9956, grad_fn=<MseLossBackward0>)
tensor(0.0263, grad_fn=<MulBackward0>)
tensor(102.9308, grad_fn=<MseLossBackward0>)
tensor(0.0019, grad_fn=<MulBackward0>)
tensor(129.6376, grad_fn=<MseLossBackward0>)
tensor(0.0278, grad_fn=<MulBackward0>)
tensor(98.9569, grad_fn=<MseLossBackward0>)
tensor(0.0021, grad_fn=<MulBackward0>)
tensor(103.7599, grad_fn=<MseLossBackward0>)
tensor(0.0026, grad_fn=<MulBackward0>)


KeyboardInterrupt: 